# YAML Template manipulations

Below a few examples and explanation for the code that will eventually be implemented in our scripts to inject arbitrary environment variables into the YAML CF template and, when deployed, to the container(s).

In [13]:
from itertools import zip_longest

import json
import yaml
import os
import re

Some constants to make working on templates and configuration files easier:

In [22]:
TEMPLATE = 'sample-template.yml'
SIMPLE_ENV = 'simple.env'
OUT = 'template-updated.yml'
VREP_ENV = 'vrep-example.env'

# RegEx pattern for the key = value parsing.
KEY_VALUE_PATTERN = re.compile(r'^\s*(\S.*)\s*=\s*(\S.*)$')

# RegEx pattern for OS Env substitution.
ENV_VAR_PATTER = re.compile(r'\${(?P<name>\S+)}')

This is what the simple configuration (`simple.env`) file looks like:
```
# Demo file for YAML manipulation

var = val

# A comment:
var2=val2


     var3    =val3
```


## Convenience methods:

In [23]:
def line2tuple(line):
    if not re.match(KEY_VALUE_PATTERN, line):
        raise ValueError("{} does not match a `key = value` pattern".format(line))
    key, value = line.split('=')
    match = re.match(ENV_VAR_PATTER, value.strip())
    if match:
        value = os.getenv(match.group('name'), value)
    return key.strip(), value.strip()

def load_env(envfile):
    """ Reads in a file listing key/value pairs into a dict
    
        Each line in the file is assumed to be a single `key = value`
        pair; leading and trailing spaces are ignored; and continuation lines
        are currently __not__ supported.
        
        In other words:
        
        ```
         key = a long long, really \
           really, long value
        ```
        
        will __not__ work.
        
        :param envfile: the name of the file to parse
        :type envfile: str
        
        :return: a list of Name/Value dicts, compatible with CloudFormation
            templates format
        :rtype: list of dict
    """
    with open(envfile) as env:
        kv = [line2tuple(line.strip()) for line in env.readlines() \
              if len(line.strip()) and not line.startswith('#')]
    return [dict(zip_longest(['Name', 'Value'], pair)) for pair in kv]

# We can read a file and generate a dictionary
# compatible with the CF Template format like this:
print(load_env(SIMPLE_ENV))

# Example of how OS Env vars are found:
match = re.match(ENV_VAR_PATTER, "${HOME}")
print(match.groups(), os.getenv(match.group('name'), '-'))

[{'Value': 'val', 'Name': 'var'}, {'Value': 'val2', 'Name': 'var2'}, {'Value': 'val3', 'Name': 'var3'}]
('HOME',) /Users/mmassenzio


## Load YAML file and parse it

In [24]:
with open(TEMPLATE) as ym:
    data = yaml.load(ym)

In [25]:
assert len(data['Resources']['taskdefinition']['Properties']['ContainerDefinitions']) == 1
ctr_defs = data['Resources']['taskdefinition']['Properties']['ContainerDefinitions'][0]

yaml_env = ctr_defs.get('Environment')

print('Environment:', json.dumps(yaml_env, indent=4), sep='\n')

Environment:
[
    {
        "Value": {
            "Ref": "ImagePort"
        },
        "Name": "SERVER_PORT"
    }
]


## Modify the environment

This simply requires loading the values from the file and appending them to the existing ones already in the template.

`TODO: parse existing ones and replacing those whose "Name" matches`

`TODO: OS env variables substitution, e.g., ${USER}`

`TODO: CLI args subsitution, e.g., #port with the value of --port`

In [26]:
# Set some env vars just to prove this does work
#
# NOTE - values MUST be strings, even if the values are numbers.
os.environ['PORT'] = '30395'
os.environ['BUILD'] = '17K4-b443ef8'

env_vars = load_env(VREP_ENV)
ctr_defs['Environment'] = yaml_env + env_vars

print(json.dumps(yaml_env + env_vars, indent=4, sort_keys=True))

[
    {
        "Name": "SERVER_PORT",
        "Value": {
            "Ref": "ImagePort"
        }
    },
    {
        "Name": "PROPERTIES",
        "Value": "aws-applicator.properties"
    },
    {
        "Name": "VREP_HOME",
        "Value": "/opt/replicator"
    },
    {
        "Name": "server.port",
        "Value": "30395"
    },
    {
        "Name": "VERSION",
        "Value": "1.6.0"
    },
    {
        "Name": "BUILD",
        "Value": "17K4-b443ef8"
    },
    {
        "Name": "HEALTHCHECK",
        "Value": "/admin/healthcheck"
    },
    {
        "Name": "user.home",
        "Value": "/Users/mmassenzio"
    }
]


In [27]:
# Put it back in the template.
data['Resources']['taskdefinition']['Properties']['ContainerDefinitions'] = [ctr_defs]

# Write out the template to disk.
with open(OUT, 'wt') as template_out:
    yaml.dump(data, template_out)

## Full (modified) template

This is in the `OUT` file:

In [28]:
with open(OUT) as template_out:
    generated = template_out.read()

pos = generated.find('ContainerDefinitions')

print('...\n', generated[pos:pos+1000], '\n...')

...
 ContainerDefinitions:
      - Cpu: {Ref: ContainerCpu}
        Environment:
        - Name: SERVER_PORT
          Value: {Ref: ImagePort}
        - {Name: PROPERTIES, Value: aws-applicator.properties}
        - {Name: VREP_HOME, Value: /opt/replicator}
        - {Name: server.port, Value: '30395'}
        - {Name: VERSION, Value: 1.6.0}
        - {Name: BUILD, Value: 17K4-b443ef8}
        - {Name: HEALTHCHECK, Value: /admin/healthcheck}
        - {Name: KAFKA, Value: 'tcp://node0.example.com'}
        - {Name: user.home, Value: /Users/mmassenzio}
        Essential: 'true'
        Image: {Ref: Image}
        LogConfiguration:
          LogDriver: awslogs
          Options:
            awslogs-group: {Ref: CloudwatchLogsGroup}
            awslogs-region: {Ref: 'AWS::Region'}
            awslogs-stream-prefix: {Ref: 'AWS::StackName'}
        Memory: {Ref: ContainerMemory}
        MountPoints:
        - {ContainerPath: /dev/xvda1, SourceVolume: ebs-vol1}
        Name: {R 
...
